# Mobile App for Lottery Addiction
In this project, I will be developing the logic for a mobile app that helps lottery addicts better estimate their chances of winning. The app will focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and enable users to answer questions like:
* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

This project will use data from the national 6/49 lottery game in Canada. The [data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,655 drawings, dating from 1982 to 2018.

## Core Functions
Throughout this project, I will need to calculate repeatedly probabilities and combinations. As a consequence, I'll start by writing two functions:
* A function that calculates factorials
* A function that calculates combination

In [4]:
# Factorial function
def factorial(n):
    answer = 1
    for i in range(n):
        answer *= i+1
    return answer

# Permutation function
def permutation(n,k):
    answer = 1
    for i in range(n,n-k,-1):
        answer *= i
    return answer

# Function calculates number of combinations when
# taking k objects from group of n objects
def combinations(n,k):
    perm = permutation(n,k)
    return perm / factorial(k)

# One-ticket Probability
For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket. For each ticket a player chooses six numberse out of 49. The next function to write will therefore be one that calculates this probability. The following details must be considered for the function:
* Inside the app, the user inputs six different numbers from 1 to 49
* Under the hood, the six numbers will come as a Python list, which will serve as the single input to the function
* The function prints the probability value in a user-friendly way - where people without any probability training are able to understand

In [50]:
def one_ticket_probability(nums):
    n_outcomes = combinations(49,6)
    prob = 1 / n_outcomes
    percentage = prob * 100
    print(f'Your chances to win the big prize with the numbers {nums} are {percentage:.5f}%.\nIn other words, you have a 1 in {int(n_outcomes):,} chances to win.')

Testing this function, it would look like this.

In [51]:
ticket = [38,39,41,9,19,21]
one_ticket_probability(ticket)

Your chances to win the big prize with the numbers [38, 39, 41, 9, 19, 21] are 0.00001%.
In other words, you have a 1 in 13,983,816 chances to win.


## Historical Data Check for Canada Lottery
The first version of the app will also allow users to be able to compare their ticket against the historical lottery data in Canada up to 2018, and determine whether they would have ever won by now. This will use the data set mentioned in the [introduction](#mobile-app-for-lottery-addiction).

In [27]:
# Import necessary modules
import pandas as pd

# Read and view data set
lottery = pd.read_csv('Datasets/649.csv')
lottery.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [28]:
print(lottery.shape)

(3665, 11)


Now that the data has been loaded and explored, I can write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now. The following details must be considered for the functions:
* Inside the app, the user inputs six different numbers from 1 to 49
* Under the hood, the six numbers will come as a Python list and serve as an input to the function
* Prints the number of times the combination selected ocurred in the Canada data set
* The probability of winning the big prize in the next drawing with that combination

In [35]:
# Add column that stores entire ticket
def add_ticket(row):
    ticket = []
    for i in range(6):
        ticket.append(row[f'NUMBER DRAWN {i+1}'])
    return sorted(ticket) #ensures order is irrelevant

lottery['TICKET'] = lottery.apply(add_ticket, axis=1)

lottery.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER,TICKET
0,649,1,0,6/12/1982,3,11,12,14,41,43,13,"[3, 11, 12, 14, 41, 43]"
1,649,2,0,6/19/1982,8,33,36,37,39,41,9,"[8, 33, 36, 37, 39, 41]"
2,649,3,0,6/26/1982,1,6,23,24,27,39,34,"[1, 6, 23, 24, 27, 39]"
3,649,4,0,7/3/1982,3,9,10,13,20,43,34,"[3, 9, 10, 13, 20, 43]"
4,649,5,0,7/10/1982,5,14,21,31,34,47,45,"[5, 14, 21, 31, 34, 47]"


In [52]:
# Check if user's ticket has won before
def historical_check(ticket):
    count = 0 # instantiate count 
    nums = sorted(ticket) # ensures order is irrelevant
    tickets = lottery['TICKET']
    for t in tickets:
        if nums == t:
            count += 1 # Loops through all tickets and counts matches
    
    # Outputs suitable message
    if count == 0:
        print(f'Your chosen numbers of {ticket} has never won the big prize.')
    elif count == 1:
        print(f'Your chosen numbers of {ticket} has won the big prize {count} time.')
    else:
        print(f'Your chosen numbers of {ticket} has won the big prize {count} times.')
        
    one_ticket_probability(ticket)

This function will now be tested.

In [53]:
test_input = [33, 36, 37, 39, 8, 41]
historical_check(test_input)

Your chosen numbers of [33, 36, 37, 39, 8, 41] has won the big prize 1 time.
Your chances to win the big prize with the numbers [33, 36, 37, 39, 8, 41] are 0.00001%.
In other words, you have a 1 in 13,983,816 chances to win.


In [54]:
test_input = [3, 2, 44, 22, 1, 44]
historical_check(test_input)

Your chosen numbers of [3, 2, 44, 22, 1, 44] has never won the big prize.
Your chances to win the big prize with the numbers [3, 2, 44, 22, 1, 44] are 0.00001%.
In other words, you have a 1 in 13,983,816 chances to win.


## Multi-ticket Probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this may increase theri chances of winning significantly. The purpose of the app is to help them better estimate their chances of winning so the next function I'll write will allow the users to calculate the chances of winning for any number of different tickets.

The following details must be considered for the functions:
* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play)
* The function will see an integer between 1 and 13,983,816 (the maximum number of different tickets)
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.


In [62]:
def multi_ticket_probability(n):
    n_outcomes = combinations(49,6)
    while n < 1 and n > n_outcomes:
        n = input('Input valid number')
    prob = n / n_outcomes
    percentage = prob * 100
    print(f'Your chance to win the big prize with {n:,} tickets is {percentage:.5f}%.')

Now, lets test the function.

In [63]:
[1, 10, 100, 10000, 1000000, 6991908, 13983816]

[1, 10, 100, 10000, 1000000, 6991908, 13983816]

In [64]:
multi_ticket_probability(1)

Your chance to win the big prize with 1 tickets is 0.00001%.


In [65]:
multi_ticket_probability(10000)

Your chance to win the big prize with 10,000 tickets is 0.07151%.


## Less Winning Numbers
In most 6/49 lotteries, there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers. So, this function will be the next to write, with the following details needing to be considered:
* The user inputs six different numbers from 1 to 49
* The user inputs an integer between 2 and 5 that represents the number of winning numbers expected

In [88]:
# Check if exactly n numbers match winning ticket
def probability_less_6(n):
    n_outcomes = combinations(49,6)
    n_combinations = combinations(6,n)
    n_success = combinations(43, 6-n)
    total_success = n_combinations * n_success
    prob = total_success / n_outcomes
    percentage = prob * 100
    print(f'Your chances of having {n} winning numbers with this ticket are {percentage:.8f}%.')
     

Testing the function.

In [86]:
probability_less_6(5)

Your chances of having 5 winning numbers with this ticket are 0.00184499%


In [87]:
probability_less_6(3)

Your chances of having 3 winning numbers with this ticket are 1.76504039%
